# Update maps for functional groups of the IUCN Global Ecosystem typology

Scripts by José R. Ferrer-Paris

The scripts described in this document are used to:

- Check map information in the database
- Read json file with map information for each EFG
- Upload maps to mapbox

## Set-up
Load all the libraries we will need in this script:

In [1]:
import os
from pathlib import Path
from configparser import ConfigParser
import psycopg2
import pandas as pd
import json
import requests


Read configuration parameters for the connection to the current version of the database:

In [10]:
filename = Path(os.path.expanduser('~')) / ".database.ini"
section = 'psqlaws'

parser = ConfigParser()
parser.read(filename)
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

params = db

In [15]:
conn = psycopg2.connect(**params)
cur = conn.cursor()

## Update information from mapbox API

In [60]:
MBfile = Path(os.path.expanduser('~'))  / ".mapbox.upload.token"
MBuser = "jrfep"

with open(MBfile) as file:
    MBtoken = file.read().strip()

headers = {"Content-Type": "application/json"}
    
api_rqst = requests.get("https://api.mapbox.com/uploads/v1/%s" % MBuser,
                   params={'access_token': MBtoken},
                   headers=headers)
api_rqst
#text = api_rqst.text

#https://api.mapbox.com/uploads/v1/%s?access_token=%s"
#rslt <- system(sprintf("curl https://api.mapbox.com/uploads/v1/%s?access_token=%s",MB.user,MB.token),intern=T)
#mbx.data <-  fromJSON(rslt)


<Response [200]>

In [67]:
records=list()
for item in api_rqst.json():
    if item['complete']==True and item['error'] is None:
        w = item['name'].split()
        record={'code':w[2],'map_code':w[5],'map_version':w[6],
                'tileset':item['tileset'],
                'created':item['created']}
        if record not in records:
            records.append(record)


In [68]:
len(records)

51

In [69]:

upd = """INSERT INTO map_files(map_code,map_version,map_file,file_description,file_comments) 
VALUES (%s,%s,%s,%s,%s)
ON CONFLICT (map_code,map_version,map_file) DO UPDATE SET file_comments=EXCLUDED.file_comments"""
for record in records:
    map_file = 'https://studio.mapbox.com/tilesets/' + record['tileset']
    map_desc="raster map in repository"
    map_comm="GeoTIFF version uploaded to Mapbox on " + record['created']     
    cur.execute(upd,(record['map_code'],record['map_version'],map_file,map_desc,map_comm))
conn.commit()

## List of valid maps

In [70]:
qry="""
SELECT code, map_code, map_version,map_type, status, update
FROM  map_metadata
WHERE status ilike 'valid'
ORDER BY code
""";
cur.execute(qry)
valid_maps=cur.fetchall()

In [71]:

df=pd.DataFrame(valid_maps,columns=['code','map_code', 'map_version', 'map_type','status', 'update'])


In [72]:
df1=df.query("code=='T7.5'")
df1

,code,map_code,map_version,map_type,status,update
196,T7.5,T7.5.WM.nwx,v2.0,Indicative Map,valid,2021-11-19 00:19:08


In [73]:
qry="""
SELECT code, map_code, map_version, map_file, file_description, file_comments,status
FROM map_files
LEFT JOIN map_metadata
USING(map_code,map_version)
WHERE map_file ilike '%mapbox%'
ORDER BY code
""";
cur.execute(qry)
list_maps=cur.fetchall()

In [74]:
df=pd.DataFrame(list_maps,columns=['code','map_code', 'map_version','map_file', 'description', 'comment', 'status'])

In [75]:
df1=df.query("code=='T7.5' and status=='valid'")
df1

,code,map_code,map_version,map_file,description,comment,status
128,T7.5,T7.5.WM.nwx,v2.0,https://studio.mapbox.com/tilesets/jrfep.T7_5_...,raster map in repository,GeoTIFF version uploaded to Mapbox on 2022-05-...,valid


In [76]:
df1[['map_version','map_file']].values

array([['v2.0', 'https://studio.mapbox.com/tilesets/jrfep.T7_5_WM_nwx']],
      dtype=object)

Read json file

In [77]:
#config.json <- read_json(sprintf("%s/data/config/groups.json", Sys.getenv("WEBDATAREPO")))
web_config = Path(os.environ['WEBDATAREPO']) / 'data/config/groups.json'
with open(web_config) as f:
    config_json = json.load(f)



In [78]:
config_json[100]

{'id': 'T7.3',
 'path': 't_7_3',
 'biome': 'T7',
 'layer': {'source': 'mapbox',
  'type': 'raster',
  'tileset': 'jrfep.T7_3_web_alt',
  'extent': {'N': '90', 'S': '90', 'W': '180', 'E': '180'}},
 'title': {'en': 'Plantations'},
 'image': {'credit': {'en': 'Prashant Y on Flickr, CC BY 2.0'},
  'caption': {'en': 'Multi-species plantation (shade coffee), Chikmagalur, India'}}}

In [79]:
for r in config_json:
    efg=r['id']
    if 'layer' in r.keys() and r['layer']['source']=='mapbox':
        old_tileset = r['layer']['tileset']
        df1=df.query("code=='%s' and status=='valid'" % efg)
        new_url = df1[['map_file']].values
        if len(new_url)==1:
            new_tileset = new_url[0][0].split("/")[4]
            if new_tileset!=old_tileset:
                print("Tileset changed from %s to %s for %s" % (old_tileset,new_tileset,efg))
                r['layer']['tileset']=new_tileset
                

In [80]:
config_json[100]

{'id': 'T7.3',
 'path': 't_7_3',
 'biome': 'T7',
 'layer': {'source': 'mapbox',
  'type': 'raster',
  'tileset': 'jrfep.T7_3_web_alt',
  'extent': {'N': '90', 'S': '90', 'W': '180', 'E': '180'}},
 'title': {'en': 'Plantations'},
 'image': {'credit': {'en': 'Prashant Y on Flickr, CC BY 2.0'},
  'caption': {'en': 'Multi-species plantation (shade coffee), Chikmagalur, India'}}}

In [81]:
#json_object =json.dumps(config_json, indent = 2).encode('utf8')
with open(web_config,"w", encoding='utf8') as f:
    json.dump(config_json, f, ensure_ascii=False, indent = 2)
    #f.write(json_object)
#with open('filename', 'w', encoding='utf8') as json_file:
#    json.dump("ברי צקלה", json_file, ensure_ascii=False)

## That's it

Let's close the connection:

In [14]:

cur.close()
        
if conn is not None:
    conn.close()
    print('Database connection closed.')

Database connection closed.
